In [ ]:
!python --version
import tensorflow as tf
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#!unzip /content/gdrive/MyDrive/File Name

In [1]:
#Imports

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Input, Lambda
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
from glob import glob
import matplotlib.pyplot as plt
import warnings
import PIL
import cv2
%matplotlib inline

In [2]:
#Image Directeories

train_path='/content/gdrive/MyDrive/...'
test_path='/content/gdrive/MyDrive/....'

In [3]:
IMAGE_SIZE = [224, 224]

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=train_path, target_size=IMAGE_SIZE, classes=['FOLDER NAME_1','FOLDER NAME_2',...], batch_size=64)

test_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=test_path, target_size=IMAGE_SIZE, classes=['FOLDER NAME_1','FOLDER NAME_2', ....], batch_size=64, shuffle=False)

In [4]:
#VGG Model

vgg = VGG16(input_shape=IMAGE_SIZE+[3], weights='imagenet',include_top=False)

In [ ]:
# don't train existing weights and train with our input image weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('/content/gdrive/MyDrive/Master Train folder Name-similar to given in train path/*')
(folders)

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
#Try wtith different Optimizer for better results

model.compile(
  loss='categorical_crossentropy',
  optimizer='Adam',
  metrics=['accuracy']
)

In [ ]:
#Model Training Phase

r = model.fit(
  x=train_batches,
  validation_data=test_batches,
  epochs=50,
  steps_per_epoch=len(train_batches)
)

In [ ]:
score = model.evaluate(test_batches,verbose=0)
print('Test Loss: ',score[0])
print('Test accuracy: ',score[1])
print(score)

In [ ]:
print(r.history.keys())

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='test loss')
plt.legend()
plt.show()
plt.savefig('Train_Test_loss')

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='test acc')
plt.legend()
plt.show()
plt.savefig('Train_Test_acc')

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)
np.round(predictions)
np.argmax(predictions, axis=-1)

In [ ]:
#Confusion Matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

cm_plot_labels = ['Folder Name_1' , 'Folder Nmae_2...']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
#AUC CURVE

from sklearn.metrics import roc_auc_score

y_prob = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

macro_roc_auc_ovo = roc_auc_score(test_batches.classes, y_prob, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    test_batches.classes, y_prob, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(test_batches.classes, y_prob, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    test_batches.classes, y_prob, multi_class="ovr", average="weighted"
)
print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)

In [ ]:
# Precison, Recall and F1 Score of Classification
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(test_batches.classes, np.argmax(predictions, axis=-1), average='weighted')